# import package

The dataset used in this notebook is Tesla stock history from 2014 to 2017. You can find the .csv file in the project folder.

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.preprocessing import StandardScaler
%matplotlib inline

### Step 0. Loading dataset

In [ ]:
tesla_stocks = pd.read_csv('data/tesla_stocks.csv')

In [ ]:
#資料共有六個資訊，日期、開盤、當日最高、當日最低、收盤、成交量
tesla_stocks.head()

In [ ]:
#簡化預測複雜度，我們只使用收盤價預測
data_to_use = tesla_stocks['Close'].values

In [ ]:
#資料共有756天成交紀錄
print('Total number of days in the dataset: {}'.format(len(data_to_use)))

### Step 1. Data preprocessing

#### Step 1.1 Scaling data

In [ ]:
#使用sklearn套件將資料標準化(mean = 0, std = 1)
scaler = StandardScaler()

In [ ]:
scaled_dataset = scaler.fit_transform(data_to_use.reshape(-1, 1))

In [ ]:
print("std: ", scaled_dataset.std())
print("mean: ", scaled_dataset.mean())

In [ ]:
tesla_stocks.Date = pd.to_datetime(tesla_stocks.Date)
plt.figure(figsize=(12,7), frameon=False, facecolor='brown', edgecolor='blue')
plt.title('Scaled TESLA stocks from August 2014 to August 2017')
plt.xlabel('Days')
plt.ylabel('Scaled value of stocks')
plt.plot(tesla_stocks.Date, scaled_dataset, label='Stocks data')
plt.legend()
plt.show()

# Config 

In [ ]:
#參數設定
learning_rate=0.001
batch_size=8
epochs = 200
hidden_layer_size=512
number_of_layers=1
dropout=True
dropout_rate=0.8
number_of_classes=1
window_size=7

In [ ]:
#抓取window_size的資料作為觀察資料(x), 下一天作為預測資料(y)
def window_data(data, window_size):
    X = []
    y = []
    
    i = 0
    while (i + window_size) <= len(data) - 1:
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
        
        i += 1
    assert len(X) ==  len(y)
    return X, y

#### Step 1.2 Windowing the dataset

In [ ]:
X, y = window_data(scaled_dataset, window_size)

#### Step 1.3 Creating Training and Testing sets

In [ ]:
#將前700筆作為訓練資料，700~749作為測試資料
X_train  = np.array(X[:700])
y_train = np.array(y[:700])

X_test = np.array(X[700:])
y_test = np.array(y[700:])

#X shape (700, 7, 1) 700筆資料, 每一筆資料有七個close price
#y shape (700, 1) 700筆資料, 每一筆資料有一個close price
print("X_train size: {}".format(X_train.shape))
print("y_train size: {}".format(y_train.shape))
print("X_test size: {}".format(X_test.shape))
print("y_test size: {}".format(y_test.shape))

### Let's create the RNN

In [ ]:
import tensorflow as tf

In [ ]:
#hidden_layer_size 是LSTM內neuron的數量
#若想堆疊堆多層LSTM使用tf.contrib.rnn.MultiRNNCell
#LSTM起始時init_state內沒有資料，先給初始值0
def LSTM_cell(hidden_layer_size, batch_size, number_of_layers, dropout=True, dropout_rate=0.8):
    def get_LSTM(hidden_layer_size, dropout, dropout_rate):
        layer = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size)

        if dropout:
            layer = tf.contrib.rnn.DropoutWrapper(layer, output_keep_prob=dropout_rate)
            
        return layer
    
    cell = tf.contrib.rnn.MultiRNNCell([get_LSTM(hidden_layer_size, dropout, dropout_rate) for _ in range(number_of_layers)])

    init_state = cell.zero_state(batch_size, tf.float32)

    return cell, init_state

In [ ]:
# outputs_shape (batch_size, timesteps, LSTM_units)
# 將lstm_output最後的輸出值再經過一層hidden layer後輸出
# 取最後一個時間點LSTM的輸出值[:, -1, :]

def output_layer(lstm_output, in_size, out_size):
    x = lstm_output[:, -1, :]
    output = tf.layers.dense(inputs= x, units= out_size)
    return output

In [ ]:
# RNN及LSTM會有梯度爆炸的問題，因此若斜率超過+-5則clip到+-5之內
def opt_loss(logits, targets, learning_rate):
    
    loss = tf.reduce_sum(tf.pow(logits - targets, 2))/batch_size
    
    #Cliping the gradient loss
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss)

    capped_gradients = [(tf.clip_by_value(grad, -5, 5), var) for grad, var in gradients if grad is not None]
    
    train_optimizer = optimizer.apply_gradients(capped_gradients)

    
    return loss, train_optimizer

### Tensorflow- 建立靜態圖 

**靜態圖**就像一張計畫圖一樣，定義我們的計算流程。實際運算必須靠 **<span style="color:red;"> Session </span>** 來執行

In [ ]:
main_graph = tf.Graph()
sess = tf.Session(graph=main_graph)

with main_graph.as_default():
    
    ##defining placeholders##
    with tf.name_scope('input'):
        inputs = tf.placeholder(tf.float32, [None, window_size, 1], name='input_data')
        targets = tf.placeholder(tf.float32, [None, 1], name='targets')
    
    ##LSTM layer##
    with tf.variable_scope("LSTM_layer"):
        cell, init_state = LSTM_cell(hidden_layer_size, tf.shape(inputs)[0], number_of_layers, dropout, dropout_rate) 
        outputs, states = tf.nn.dynamic_rnn(cell, inputs, initial_state=init_state)
    
    ##Output layer##   
    with tf.variable_scope('output_layer'):
        logits = output_layer(outputs, hidden_layer_size, number_of_classes)
    
    ##loss and optimization##
    with tf.name_scope('loss_and_opt'):
        loss, opt = opt_loss(logits, targets, learning_rate)
    
    init = tf.global_variables_initializer()
    

### Tensorflow- 初始化模型

In [ ]:
#### initialize model ####
sess.run(init)

### Tensorflow- 實際執行模型訓練

In [ ]:
for i in range(epochs):
    ii = 0
    epoch_loss = []
    while(ii + batch_size) <= len(X_train):
        X_batch = X_train[ii:ii+batch_size]
        y_batch = y_train[ii:ii+batch_size]
        batch_loss, _ = sess.run([loss, opt], feed_dict={inputs:X_batch, targets:y_batch})
        epoch_loss.append(batch_loss)
        ii += batch_size
    if (i % 30) == 0:
        print('Epoch {}/{}'.format(i, epochs), ' Current loss: {}'.format(np.mean(epoch_loss)))

In [ ]:
#Training set預測結果
training_set_pred = np.array([])
for i in range(len(X_train)):
    o = sess.run(logits, feed_dict={inputs:X_train[i:i+1]})
    training_set_pred = np.append(training_set_pred, o)

In [ ]:
#Testing set預測結果
testing_set_pred = np.array([])
for i in range(len(X_test)):
    o = sess.run(logits, feed_dict={inputs:X_test[i:i+1]})
    testing_set_pred = np.append(testing_set_pred, o)    

In [ ]:
#把資料放到list裡面準備畫圖
#因為我們是用前七天預測第8天股價，故前七天設為None
training = [None]*window_size
for i in range(len(X_train)):
    training.append(training_set_pred[i])
testing = [None] * (window_size + len(X_train))
testing_loss = 0
for i in range(len(X_test)):
    testing.append(testing_set_pred[i])
    testing_loss += (testing_set_pred[i] - y_test[i])**2
    training.append(None)
print('testing loss:', testing_loss / len(X_test))

### Plotting predictions from the network

In [ ]:
plt.figure(figsize=(16, 7))
plt.plot(tesla_stocks.Date, scaled_dataset, label='Original data')
plt.plot(tesla_stocks.Date, training, label='Training data')
plt.plot(tesla_stocks.Date, testing, label='Testing data')
plt.legend()
plt.show()

In [ ]:
sess.close()

## 練習
1. 把Open、High、Low、Close跟Volume都當Feature